**Содержание**<a id='toc0_'></a>    
- [Стандарты C++11 и C++14](#toc1_)    
  - [Стандартизация C++](#toc1_1_)    
  - [Основные принципы разработки стандарта](#toc1_2_)    
  - [Мелкие улучшения C++14](#toc1_3_)    
  - [nullptr](#toc1_4_)    
  - [Вывод типов](#toc1_5_)    
  - [Альтернативный синтаксис для функций](#toc1_6_)    
  - [Шаблоны с переменным числом аргументов](#toc1_7_)    
  - [Ключевые слова default и delete](#toc1_8_)    
  - [Делегация конструкторов](#toc1_9_)    
  - [Явное переопределение и финальность](#toc1_10_)    
- [Семантика перемещения](#toc2_)    
  - [Излишнее копирование](#toc2_1_)    
  - [Перемещающий конструктор и перемещающий оператор присваивания](#toc2_2_)    
  - [Перемещающие методы при помощи swap](#toc2_3_)    
  - [Задача](#toc2_4_)    
  - [Использование перемещения](#toc2_5_)    
  - [Когда вызываются перемещающие методы](#toc2_6_)    
  - [Перегрузка с lvalue/rvalue](#toc2_7_)    
  - [Перемещающие особые методы](#toc2_8_)    
  - [Пример: unique_ptr](#toc2_9_)    
- [Ещё о нововведениях C++11 и C++14](#toc3_)    
  - [Кортежи](#toc3_1_)    
  - [Константные выражения constexpr](#toc3_2_)    
  - [Range-based for](#toc3_3_)    
  - [Списки инициализации](#toc3_4_)    
  - [Универсальная инициализация](#toc3_5_)    
  - [std : : function](#toc3_6_)    
  - [Лямбда-выражения](#toc3_7_)    
  - [Различные виды захвата](#toc3_8_)    
  - [Новые строковые литералы](#toc3_9_)    
  - [Перечисления со строгой типизацией](#toc3_10_)    
- [Как работают rvalue-ссылки](#toc4_)    
  - [Преобразование ссылок в шаблонах](#toc4_1_)    
  - [Как работает std : : move?](#toc4_2_)    
  - [std : : move для lvalue](#toc4_3_)    
  - [std : : move для rvalue](#toc4_4_)    
  - [Perfect forwading](#toc4_5_)    
  - [Как работает std : : forward?](#toc4_6_)    
  - [std : : forward для lvalue и rvalue](#toc4_7_)    
  - [Variadic templates + rvalue reference](#toc4_8_)    
  - [Резюме по std::move и std::forward](#toc4_9_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Стандарты C++11 и C++14](#toc0_)

## <a id='toc1_1_'></a>[Стандартизация C++](#toc0_)

- 1983 Появление С++
- 1998 С++98 - первый стандарт
- 2003 С++03 - исправление недостатков С++98
- 2011 С++11 - многопоточность
- 2014 С++14 - исправления недостатков С++11
- 2017 С++17 - развитие С++14
- 2020 C++20 - развитие С++17 (модули в дополнение к `#include`, сопрограммы, форматировние строк в стиле Python, `<ranges>`, `<chrono>` и др.)

## <a id='toc1_2_'></a>[Основные принципы разработки стандарта](#toc0_)

- сохранение совместимости с предыдущими версиями
- улучшение техники программирования
- улучшение С++ с т.з. дизайна
- увеличение типобезопасности
- увеличение производительности
- исключение затрат ресурсов на возможности языка, которые не используются
- расширение возможностей через расширение стандартной библиотеки, а не через ядро языка
- упрощение языка для изучения/освоения при сохранении возможностей экспертного уровня

## <a id='toc1_3_'></a>[Мелкие улучшения C++14](#toc0_)

1. Исправление проблемы с угловыми скобками: `T<U<int>>`
- `>>` воспринималось как оператор сдвига, и нужно было добавлять пробел
2. Понятия "тривиального класса" и "класса стандартным размещением"
- низкоуровневая работа с объектами: побайтное копирование, сериализация и т.п. были доступны только для `struct`, теперь для классов, удовлетворяющих опред.условиям
3. `explicit operator bool () { ... }` вместо корявой идеомы safe bool (см. выше)
- до этого было нельзя использовать `explicit` в операторах такого рода
- нужно, чтобы приводить объект к bool, но при этом нельзя было бы сравнивать больше / меньше, только проверить истинность
4. Шаблонный `typedef` (в виде `using`)
- до этого `typedef` можно было применять только для завершенных типов, у которых можно вызвать `sizeof`
```
    template< class A, class B, int N>
    class SomeType { ... };

    template<typename B>
    using TypedefName = SomeType<double, B, 5>;
```
- как бы такой синоним для полуопределенных шаблонов, где часть типов/аргументов подставляется заранее (в примере `double`,  `5`)
- аналогично можно задачать синонимы для типов указателей на функции
```
    typedef void (*OtherType)(double);   // старый синтаксис
    using OtherType = void (*) (double); // новый (более читабельный)
```

**Еще пример:**

Перепишите на `using` следующий `typedef`.

    typedef int * (Class::*Foo)(int, double) const;
    using Foo = int * (Class::*)(int, double) const;

5. Новый тип `long long int` 
- 64-битный инт, причем доступный и на 32-битных платформах
- этот тип давно есть в C, но не все что есть в С, автоматически есть в С++ (С99 не подмножество С++), теперь этот тип из С есть и в С++
6. Библиотека поддержки типов `<type_traits>`
- на этапе компиляции по типу можно узнавать его свойства и управлять компиляцией
    - в шаблонной функции можно проверить, что прилетел целочисленный тип или еще какой, поднастроить шаблон под него
    - можно изменять некоторые типы, например из указателя на тип получить тип

7. Новые операторы выравнивания данных `alignof` и `alignas`
- для каких-то операций требуется строгое выравнивание данных в памяти, например ровно по 4 байта (т.е. адрес должен делиться на 4 без остатка), иначе результат операции будет неверный
- например `alignas (float) unsigned char c[sizeof(float)];` выделит массив из 4 байт так, что их вместе можно будет интерпретировать как `float` без ошибок
8. В дополнение к проверкам в рантайме (`assert`) добавился новый оператор `static_assert` для проверок условий на этапе компиляции (если не выполняется, то компилироваться не будет). Например, проверка, что тип Т знаковый (`is_signed` из `<type_traits>`):

```
    template <class T>
    void run(T * data, size_t n) {
        static_assert(std::is_signed<T>::value, "T is not signed type.")
    }
```

**Задача**

Предположим, что класс String имеет оператор приведения к `char const *`.

    struct String
    {
        // обычная реализация строки
       ...

        // оператор приведения к char const *
        operator char const *() const;
    };

Выберите те строки, которые перестанут компилироваться, если этот оператор объявить как `explicit`.

    String s("Hello");                              // это не указатель, а значение!
    delete s;                                       // 1 неявно к указателю
    if (s) { }                                      // 2 неявно к bool
    char const * p1 = s;                            // 3 неявно к указателю
    char const * p2 = (char const*)s;               // 4 ОК (отработает оператор класса)   
    char const * p3 = static_cast<char const*>(s);  // 5 ОК (отработает оператор класса)
    char const * s2 = s + 4;                        // 6 сначала неявно к байтам (?), потом неявно к указателю

Останутся рабочими только 4 и 5, т.к. в остальных неявное приведение типа, которое `explicit` не позволит.

## <a id='toc1_4_'></a>[nullptr](#toc0_)

Новый литерал, представляющий тип `std::nullptr_t`. Объекты этого типа могут иметь единственное возможное значение (`nullptr`), которое неявно приводятся к нулевому указателю на любой тип:

    void foo(int a) { ... };

    void foo(int * p) { ... };

    void bar() {
        foo(0);             // по перегрузке будет выбран вызов foo(int)
        foo((int *) 0);     // foo(int*) С++98 явное приведение к 0 указателю на int
        foo(nullptr);       // foo(int*) С++11 сразу передаем нулевой указатель, который неявно приведется к 0 указателю на int 
    }

Нужно, чтобы избежать неоднозначности 0 как числа 0, и как 0 указателя. Решение с макросом `NULL` тоже не лишено недостатков (он может быть по разному определен в разных версиях, как 0 или как `(void*)0` или еще как, что тоже может добавить тонкостей)

## <a id='toc1_5_'></a>[Вывод типов](#toc0_)

Два новых механизма: `auto` и `decltype` (работают "статически", только на этапе компиляции, к RTTI отношения не имеют)
```
    Array<Unit*> units;

    for (size_t i = 0; i != units.size(); ++i) {
        // Unit *
        auto u = units[i];

        // Array<Item> const & (допустим такой тип возвращает метод items(), из кода это никак не следует)
        decltype(u->items()) items = u->items();
        ...
    }
```

Ключевое слово `auto` (было и в С++11, но совсем с другим смыслом и практически не использовалось):
- автоматический вывод типа переменной на основе выражения справа от знака `=`
    - компилятор видит, что это массив `Unit`, значит тип `u` - это `Unit*`
    - в данном примере это очень очевидно, но на практике так удобно выводить типы из сложных шаблонов от шаблонов, компилятор тут не будет ошибаться, а человек может
- в С++17 к `auto` также добавили возможность распаковки аргументов в стиле Python/JS
    - `auto [a, b, c] = object;` если объект можно разложить на 3 составляющие (или другое количество), то они положатся в `a`, `b`, `c`

Оператор `decltype`:
- вычисляет и возвращает тип выражения в скобках. Делает это более подробно, чем `auto`, т.е. покажет константность и ссылочность типа
- в данном случае тут мог бы быть и `auto`, который вернул бы тип просто `Array<Item>`
    - если с `auto` нужны более строгие квалификторы типа, то нужно писать их ручками типа такого `const auto & a = items[0];`

**Примеры и тонкости**

    auto a = items[0];          // a - Item (т.к. это просто тип, то тут произойдет копирование элемента)
    
    decltype(items[0]) b = a;   // b - Item const & (т.к. выше мы получили в items массив константных ссылок, aka ссылки на константные объекты, т.е. [] тут - это константный вариант оператора) - а т.к. это ссылка, то тут копирования не произойдет, просто захват ссылки на константный элемент

В `decltype` есть особая форма, когда он применяется просто к имени переменной, а не выражению, то он возвращает просто тип, с которой была определена эта переменная

    decltype(a) c = a;          // c - Item (т.к. a имела такой тип при определении)
    decltype((a)) d = a;        // d - Item & (это уже от выражения, тут тип вычисляется и получаем ссылку на `a`)

    decltype(b) e = b;          // e - Item const & (тут будет одинаково от имени и выражения, т.к. они получили одинаковый тип при создании)
    decltype((b)) f = b;        // f - Item const &

**Замечания**
1. Константная ссылка - это такое корявое устоявшееся выражение. Ссылка сама по себе неизменяемая, поэтому это означает ссылку на константный объект.
2. По аналогии с оператором `sizeof` оператор `decltype` не вычисляет реальный тип, который окажется тут в рантайме, а только жонглирует определениями типов, которые ему известны выше по коду. Если массив `units` из примера будет длины 10, а мы объявим `decltype(items[100]) b = a;`, то результат будет такой же

**Задача**

Выберите правильные утверждения про следующий код.

    std::string s = "Compiler";

    auto sz = s.size();

    auto c1 = s[0];

    decltype(s[0]) c2 = s[7];

    decltype(s[0] + s[7]) c3 = s[1];

    decltype(s)   sa = s;

    decltype((s)) sb = s;

    c1 = 'K';
    c2 = '!';

Переменная sz имеет тип size_t
- да, хотя `typeid` говорит, что это `unsigned long`
Переменная c1 имеет тип char
- да, т.к. `auto` дает просто тип (почему?)

Переменная c2 имеет тип char
- **нет**, `s[0]` это выражение, поэтому тип будет `char &`

Переменная c3 имеет тип int
- да, просто такое правило языка, при сложении целочисленных меньше `int` они приводятся к `int` (потому что для `int` определен оператор +)

В этом коде создаётся только два объекта std::string
- да, при определении `sa`, т.к. это не выражение, а переменна, будет произведено копирование `s`

После выполнения этого кода `s` хранит строку "Kompile!"
- **нет**, т.к. `c1` не `char &`, а просто `char`, то присвоение ему значения `'K'` не изменит `s`. В `s` будет `Compile!`

## <a id='toc1_6_'></a>[Альтернативный синтаксис для функций](#toc0_)

В стандарте С++11 / С++14 для повышения удобства написания кода появился очередной новый "еще более удобный" синтаксис для функций.

Допустим, мы хотим написать функцию для сложения двух аргументов и хотим сделать ее универсальной, т.е. зашаблонить:

    template <typename A, typename B>
    RETURN_TYPE Plus(A a, B b) { return a + b; }

Очевидно, в `A` и `B` должны быть определены `operator +`, но какой должен быть `RETURN_TYPE`? Ведь этот тип зависит от логики реализации сложения для данных типов, где-то это должен быть `A`, где-то `B`, а может и вообще какой-то третий тип:

    int + double -> double
    double + int -> double
    char + char -> int
    std::strint + char * -> std::string
    ...

Если использовать `decltype`, то ничего не выйдет:
    
    // некорректно, т.к. типы a и b будут определены позже (хоть и в той же строке, но компилятор их еще не видел)
    template <typename A, typename B>
    decltype (a + b) Plus(A a, B b) { return a + b; }

Эту проблему решает новый синтаксис - синтаксис для поясниловки для `auto` откуда брать типы. В С++14 добавили возможность не писать `-> decltype (expr)` если компилятору достаточно информации и так (функция простая и тип понятен по очевидному ретурну):

    // С++11
    template <typename A, typename B>
    auto Plus(A a, B b) -> decltype (a + b) { return a + b; }

    // С++14 используюя правила вывода auto
    template <typename A, typename B>
    auto Plus(A a, B b) { return a + b; }

    // или C++14 используюя правила вывода decltype
    template <typename A, typename B>
    decltype(auto) Plus(A a, B b) { return a + b; }

Или можно использовать старый стиль есть есть конструкторы по умолчанию:

    template <class A, class B>
    decltype(A{}+B{}) summ(A a, B b)
    {
        return a + b;
    }

Еще вариант как можно использовать старый стиль, можно заморочить вот так (тут rvalue reference, noexcept и приведение типа к ссылке в `std::declval`)

    template <class A, class B>
    decltype (std::declval<A>() + std::declval<B>()) Plus(A &&a, B &&b) noexcept
    {
        return a + b;
    }

## <a id='toc1_7_'></a>[Шаблоны с переменным числом аргументов](#toc0_)

В старом стандарте с переменным числом аргументов могли быть только функции (типа `printf` и т.п.)

Рекурсивный вариант `printf` для переменного числа аргументов
    
    // база рекурсии: нет аргументов, значит не должно оставаться и необработанных упр. символов форматной строки (кроме, возможно, %%)
    void printf(char const *s) {
        while (*s) {
            if (*s == '%' && *(++s) != '%')
                // обработка ошибки
                assert(!"unhandled format specifier");
            std::cout << *s++;
        }
    }

    // список аргументов задается так
    template<typename T, typename... Args>
    void printf(char const *s, T value, Args... args) {
        while (*s) {
            if (*s == '%' && *(++s) != '%') {
                std::cout << value;
                printf(++s, args...);
                return;
            }
        std::cout << *s++;
        }
        // обработка ошибки
        assert(!"extra arguments provided to mprintf");
    }

*) чтоб не конфликтовало с встроенным `printf` либо переименовать, либо обернуть в пространство имен
**) форматная строка не проверяет соотвествие аргументов и не умеет настраивать точность как `printf`, просто любое сочетание `%*` значит выводится следующий аргумент

**Задача**

По аналогии с функцией `printf` с предыдущего шага напишите функцию `print_values` с переменным числом аргументов, которая для каждого аргумента выводит его тип и значение в поток `std::ostream`, который ей передан в качестве первого аргумента. Функция должна работать с произвольным числом аргументов.
Указание: для вывода имени типа нужно использовать `std::type_info::name`.

    // база рекурсии
    void print_values(std::ostream & os) { };

    // Шаблон с переменным числом аргументов
    template<typename T, typename... Args>
    void print_values(std::ostream & os, T const & value, Args const &... args)
    {
        os << typeid(value).name() << ": " << value << "\n";
        return print_values(os, args...);
    }

C++17 еще проще

    template <typename ...Args>
    void print_values(std::ostream& os, const Args&... args)
    {
        ((os << typeid(args).name() << ": " << args << std::endl), ...);
    }

Фокус с оператором "запятая"

    template <typename... Args>
    void print_values(std::ostream& os, const Args&... args)
    {
        int checkers[sizeof...(Args)] = { (os << typeid(args).name() << ": " << args << std::endl, 0)... };
    }

- `z = (a, b);` вычисляет а, потом b, присваивает z значение b
- в примере - создание массива нулей длины по количеству аргументов, при этом в левой части оператора "запятая" выводятся аргументы в поток вывода

## <a id='toc1_8_'></a>[Ключевые слова default и delete](#toc0_)

У классов могут быть методы, которые компилятор может сгенерировать самостоятельно (констр.по умолчанию, констр.копирования, оператор присваивания, деструктор). Если добавить в класс любой пользовательский конструктор, то компилятор **не будет генерировать никакие свои конструкторы**.

Если мы вдруг хотим получить от компилятора свои автоматические конструкторы, несмотря на то, что добавили пользовательские, в новом С++ добавили ключевое слово `default`:

    struct SomeType {
        SomeType() = default;       // Конструктор по умолчанию от компилятора
        SomeType(OtherType value);
    };

Также добавили возможность выборочно не генерировать компилятором отдельных специальных методов (слово `delete`):

    struct NonCopyable {
        NonCopyable() = default;                    // Конструктор по умолчанию хотим
        NonCopyable(const NonCopyable&) = delete;   // Конструктор копирования НЕ хотим
        NonCopyable & operator=(const NonCopyable&) = delete;   // Оператор присваивания НЕ хотим
    };

Наверно это немного улучщает читаемость, т.к. дефолтные конструкторы все равно ничего не делают, поэтому 
    
    SomeType() = default;

это тоже самое, что написать 

    SomeType() {};

Ключевое слово `delete` можно использовать для удаления любых функций, и это может быть частью полезного паттерна:

    template<class T>
    void foo(T const * p) { ... };

    void foo(char const * p) = delete;

- шаблонная функция будет работать с любыми типами, кроме в данном случае `char` (error: use of deleted function)
- явная перегрузка функции имеет больший приоритет, чем шаблон, функция есть - берется она, а не шаблон. Но она удалена, поэтому ошибка

## <a id='toc1_9_'></a>[Делегация конструкторов](#toc0_)

**Делегирование конструкторов** - вызов одного конструктора из другого конструктора этого же класса (раньше так было низзя, приходилось выносить общий код в отдельный метод и вызывать его из разных конструкторов)

**Примеры:**

    // в конструкторе по умолчанию вызываем конструктор с параметрами с некоторыми умолчательными параметрами
    struct SomeType {
        SomeType(int newNumber): number(newNumber) {}
        SomeType() : SomeType(42) {}
    private:
        int number;
    }

    // тут пример с какими-то тонкостями и немного не про то, инициализацию по умолчанию можно прописать непосредственно в поля, а при вызове конструктра с параметрами их переприсвоить (а что, так нельзя было раньше?)
    struct SomeClass {
        SomeClass() {}
        explicit SomeClass(int newNumber): number(newNumber) {}
    private:
        int number = 5;
    }

Ранее при наследовании наследовались только методы, а конструкторы всегда надо было переписывать ручками, теперь их можно использовать через `using` (можно применять не только к конструкторам, но и к другим полям и методам класса)

    struct Base {
        Base(int value) : data(value) {};
    protected:
        void f() { std::cout << "Base::f()\n"; }
        int data = 0;
    }

    struct Derived : public Base {
        using Base::Base;
        using Base::f;
    }

- в данном примере поля и методы базового класса `protected`, поэтому обращение к ним напрямую через наследованный класс невозможно (можно только во внутренних методах класса). А использование `using` в наследованном классе здесь делает их публичными.

## <a id='toc1_10_'></a>[Явное переопределение и финальность](#toc0_)

Появилась возможность явно переопределять виртуальные методы и управлять финальностью.

```
    struct Base {
        virtual void update();
        virtual void foo(int);
        virtual void bar() const;
    }

    struct Derived : public Base {
        void updata() override;                 // (1) error
        void foo(int) override;                 // (2) OK
        virtual void foo(long) override;        // (3) error
        virtual void foo(int) const override;   // (4) error
        virtual int foo(int) override;          // (5) error
        virtual void bar(long);                 // (6) OK
        virtual void bar() const final;         // (7) OK
    }

    struct Derived2 final : Derived {
        virtual void bar() const;               // (8) error
    }
    
    struct Derived3 : Derived2 {};              // (9) error
```

1. `override` директива компилятору проверить наличие такого метода в табл.вирт.мет. - у нас опечатка в названии, он об этом скажет, а не создаст метод с новым именем
2. Все ОК, сигнатура в табл.вирт.мет. совпадает, переопределение вирт.метода в потомке ОК.
3. Сингатура
4. Сингатура
5. Константность
6. Это не преопределение метода, а создание нового вирт. Такой сигнатуры не было, поэтому ОК
7. `final` запретит в потомках переопределение вирт. метода с такой сигнатурой
- **Замечание**: такой метод есть в предке, это переопределение, поэтому лучше наприсать тут `virtual void bar() const override final`
8. Этот метод зафинален выше, низзя так
9. Запрещено наследоваться от `Derived2`


# <a id='toc2_'></a>[Семантика перемещения](#toc0_)

Набор нововведений для оптимизации перещения/копирования объектов

## <a id='toc2_1_'></a>[Излишнее копирование](#toc0_)

Проблема при возвращении объектов из функций:

    struct String {
        Srting() = default;
        String(String const & s);
        String & operator=(String const & s);
        //...
    private:
        char * data_ = nullptr;
        size_t size_ = 0;
    }


Копирование - получаем на вход строку, выделяем в дин.памяти буфер такого же размера, посимвольно копирует входную строку.

Аналогично происходит присваивание - с тем отличием, что предварительно нужно/желательно очистить текущий/старой буфер строки, если он не пустой.

Копирование/присваивание объектов типа строк - это затратная операция, пропорциональная длине строки, их надо стараться избегать.

Как происходит возврат объектов из функции:

    String getCurrentDateString() {
        String date;
        // date заполняется "21 октября 2015 года"
        return date;
    }
    
    String date = getCurrentDateString();

- При возврате по значению в `return date;` вызывается конструктор копирования из `date` на стек вызывающей функции
- При присваивании `String date =` также будет вызван конструктор копирования (т.к. в переменной `date` к этому моменту еще нет строки, что вызвало бы код оператора присваивания)

При включенных оптимизациях компилятора в данном случае , скорее всего двух копирований можно будет избежать, он заметит, что внутри функции и снаружи переменные `date` по сути одинаковые, сразу будет создана наружняя переменная и писаться будет сразу в нее,  но это не гарантируется в общем случае. 

В старом стандарте основным способом избежать этого было выделение строки в дин.памяти и возврат ее по указателю/ссылке. А это требует дополнительных действий по работе с дин.памятью (следить за освобождением или использовать умные указатели и т.п.)

## <a id='toc2_2_'></a>[Перемещающий конструктор и перемещающий оператор присваивания](#toc0_)

В новом стандарте появились два новых специальных метода (перемещающий конструктор и перемещающий оператор присваивания), направленных на решение проблем излишнего копирования объектов

    struct String
    {   
        // перемещающий конструктор
        String (String && s) // && - rvalue reference
            : data_(s.data_)
            , size_(s.size_) {
                s.data_ = nullptr;
                s.size_ = 0;
        }
        // перемещающий оператор =
        String & operator = (String && s) {
            delete [] data_;
            data_ = s.data_;
            size_ = s.size_;
            s.data_ = nullptr;
            s.size_ = 0;
            return *this;
        }
    };

**Перемещающий конструктор** по сигнатуре похож на к.копирования, только объект получается не по `const &` (обычно так), а по `&&` - это новый тип ссылок (**rvalue ссылки** см. далее).
- к.копирования получает объект и не должен его изменять (поэтому обычно передают по `const &`)
- здесь мы должны оставить исх.объект в некотором согласованном состоянии (обычно пустом/обнуленном), поэтому
    - здесь мы отбираем данные у исходного объекта (через инициализирующие параметры `data_(s.data_), size_(s.size_)`)
    - зануляем его указатель на данные/буфер и его размер
    - теперь данными владеет только `this`

**Перемещающий оператор =** - аналогично:
- обнуляем текущий буфер/данные этого объекта (в нем могли быть старые данные, их оставаться нигде не должно) 
    - `delete [] data_` возвращает память в кучу как свободную (размер - по метаинформации, которую в начало это блока памяти записал ранее `new[]`)
- захватываем данные исходного объекта
- разрываем связь между захваченными данными и исх.объектом

**Замечание**

1. В обычном к.копирования использование `const &` "почти обязательно", чтобы можно было копировать также и константные объекты (неконстантные неявно приводятся к константным, а наоборот - нет)

2. Как и с оператором =, в перемещающем оп. = нужно проверять, что мы не перемещаем сами в себя, иначе объект "сломается" (старые данные удалятся, но они же одновременно и новые данные)
```
    if(this != &s)
    {
        // move data
    }
```

## <a id='toc2_3_'></a>[Перемещающие методы при помощи swap](#toc0_)

Хорошей / правильной реализацией оп.= является использование метода `swap`. Аналогичное верно для перемещ.оп.=:

    #include<utility>

    struct String
    {
        void swap(String & s) {
            std::swap(data_, s.data_);
            std::swap(size_, s.size_);
        }
        String (String && s) {
            swap(s);
        }
        String & operator = (String && s) {
            swap(s);
            return *this;
        }
    };

В общем случае надо обеспечить нулевые/пустые поля в конструкторе объекта до обмена с перемещаемым.

- Тут в коде мы исходим из того, что в классе `String` использовался список инициализации (из кода это не следует, но считаем, что это так), поэтому `swap` работает корректно. 
- В методе оператора перемещающего присваивания тут допущена вольность, перед `swap(s)` обязательно нужно очищать текущий объект. Иначе текущие данные будут висеть в памяти пока не отработает деструктор перемещаемого объекта или его не изменят иным способом (а если там чувствительные данные?), короче не надо так делать.
- `swap(s);` вызывается как метод текущего класса (`this->swap(s);` не обязательно)

## <a id='toc2_4_'></a>[Задача](#toc0_)

Дополните класс Array перемещающим конструктором и перемещающим оператором присваивания.

    template<class T>
    struct Array
    {
        // все объявленные ниже методы уже реализованы
        explicit Array(size_t size = 0);
        Array(Array const& a);
        Array & operator=(Array const& a);
        ~Array();

        size_t size() const;
        T &         operator[](size_t i);
        T const&    operator[](size_t i) const;

        // реализуйте перемещающий конструктор
        
        // ВАЖНО значения по умолчанию
        Array (Array && a) : size_(0), data_(nullptr) {
            swap(a);
        }
        
        // реализуйте перемещающий оператор присваивания
        Array & operator = (Array && a) {
            if (this != &arr)
                swap(a);
            return *this;
        };
        
    private:         
        size_t  size_;
        T *     data_;    

        void swap(Array & a) {
            std::swap(data_, a.data_);
            std::swap(size_, a.size_);
        };
    };

**Замечание:**
Модификаторы доступа `protected` и `private` не позволяют обращаться извне класса к полям и методам, объявленным/определенным с данными модификаторами, однако это вовсе не означает, что нельзя обращаться из объекта `foo` некоторого класса `T` к `protected/private` полям другого объекта `bar` того же класса `T`. **Модификаторы доступа действуют на уровен КЛАССА, а не ЭКЗЕМПЛЯРА!**

## <a id='toc2_5_'></a>[Использование перемещения](#toc0_)

    struct String {
        String() = default;
        String(String const & s); // lvalue-reference
        String & operator=(String const & s);
        String(String && s); // rvalue-reference
        String & operator=(String && s);
    private:
        char * data_ = nullptr;
        size_t size_ = 0;
    };

    String getCurrentDateString() {
        String date;
        // date заполняется "21 октября 2015 года"
        return std::move(date);
    }
    
    String date = getCurrentDateString();   // к.перемещ.копир.
    
    date = getCurrentDateString();          // оп.перемещ.присваивания

Принудительно заставить использовать перемещающий к.копирования/оп.= можно с пом. `std::move`

- возвращаемый по значению объект - и так `rvalue`, поэтому не обязательно `String &&  getCurrentDateString() { ... }`
- конкретно при возврате значений из функций лучше явно не вызывать `std::move`, а дать дорогу компилятору, тут чисто для примера


## <a id='toc2_6_'></a>[Когда вызываются перемещающие методы](#toc0_)

В примере выше можно было не использовать `std::move`. Дело в том, что при возвращении локального объекта функции (каким и является `date`) по значению будет вызываться перемещающий конструктор. Т.е. код функции можно было оставить без изменения.

    String getCurrentDateString()
    {
        String date;
        // date заполняется "21 октября 2015 года"

        // без std::move 
        return date; // date будет перемещена
    }

Более того, такой код потенциально работает более эффективно: так как в данном случае компилятору позволяется сделать оптимизацию возвращаемого значения (RVO).
Вот неполный список случаев, когда будут вызываться перемещающие методы:

- если передавать в них объект при помощи `std::move()`;
- если передавать в них временный объект;
- если из функции по значению возвращается локальный объект функции.

Некоторые другие случаи, в которых вызываются перемещающие методы, мы увидим в дальнейшем в этом курсе.

## <a id='toc2_7_'></a>[Перегрузка с lvalue/rvalue](#toc0_)

При перегрузке перемещающий метод вызывается для временных объектов и для явно перемещённых с помощью `std::move`.

    String a(String("Hello"));      // перемещение (tmp == rvalue)
    String b(a);                    // копирование (lvalue)
    String c(std::move(b));         // перемещение (std::move)
    a = b;                          // копирование (lvalue)
    b = std::move(c);               // перемещение (std::move)
    c = String("world");            // перемещение (tmp == rvalue)

    String s1 = String("Hello");            // перемещение (tmp == rvalue)
    String s2("world");                     // конструктор с параметром
    String s3 = s1 + ", " + s2 + "!";       // перемещение (tmp == rvalue)
    String s4 = s3;                         // копирование
    s1 = std::move(s4);                     // перемещение (std::move)
    s2 = String(s1);                        // перемещение (tmp == rvalue)

Это касается и обычных методов и функций, которые принимают lvalue/rvalue-ссылки.

`tmp` - это временные объекты, созданные прямо тут или возвращенные из функции. Они всегда `rvalue`

## <a id='toc2_8_'></a>[Перемещающие особые методы](#toc0_)

Особые методы класса (которые в новом стандарте копилятор генерирует самостоятельно):
- конструктор по умолчанию,
- конструктор копирования,
- оператор присваивания,
- деструктор,
- **перемещающий конструктор**,
- **перемещающий оператор присваивания**.

Перемещающие методы генерируются только, если в классе отсутствуют пользовательские копирующие операции, перемещающие операции и деструктор. Или если это явно указано с помощью `default`

Генерация копирующих методов для классов с пользовательским конструктором признана устаревшей. В дальнейшем они могут быть изменены на аналогичные перемещающим методам

## <a id='toc2_9_'></a>[Пример: unique_ptr](#toc0_)

Гарантирует уникальное владение указателем. Класс `unique_ptr` перегружает оператор `->`, что позволяет обращаться к полям класса и вызывать его методы, словно мы работаем с обычным указателем. При уничтожении указателя автоматически **вызывается деструктор** объекта, на который он указывает:

    #include <memory>   // std::unique_ptr
    #include "units.hpp"

    void foo(std::unique_ptr<Unit> p);

    std::unique_ptr<Unit> bar();

    int main() {
        // p1 владеет указателем
        std::unique_ptr<Unit> p1(new Elf());
    
        // теперь p2 владеет указателем
        std::unique_ptr<Unit> p2(std::move(p1));

        // std::unique_ptr<Unit> p2(p1);    // ош.компиляции (нет к.копирования)
        // std::unique_ptr<Unit> p2 = p1;   // ош.компиляции (нет оп.=)

        p1 = std::move(p2); // владение передаётся p1
        
        foo(std::move(p1)); // p1 передаётся в foo

        p2 = bar(); // std::move не нужен (объект временный, исходный указатель уничтожится при выходе из функции)

        // мы всегда можем получать из `unique_ptr` обычный указатель на объект
        Unit* ptr = p2.get();
        // или ссылку...
        auto & ref = p3; // Unit &
    }


В старом стандарте есть `std::auto_ptr`, который в операторе присваивания и к.копирования вместо ожидаемого копирования данных делал их захват, т.е. по сути выполнял перемещение (изменял тот объект, который получал в к.копирования и оп.=). Хотя семантики перемещения в старом стандарте нет...

В новом стандарте вместо него ввели `std::unique_ptr`, который делает тоже самое, но уже явно с использованием новой семантики (старый остался для совместимости, но в стандарте C++17 `auto_ptr` был удален.).

В `std::unique_ptr` удалены к.копирования и оп.=, только перемещение. В результате указателем на его клиентский класс может владеть только одна переменная.

В С++14 добавили шаблон:

    auto unq = std::make_unique<Unit>(/* ctor args */);

**Дополнительно**

Класс `std::shared_ptr` является указатем на объект, которым владеет сразу несколько объектов. Указатель можно как перемещать, так и копировать. Число существующих указателей отслеживается при помощи счетчика ссылок. Когда счетчик ссылок обнуляется, вызывается деструктор объекта. 

В `std::shared_ptr` возможны циклические ссылки (a -> b, b -> a) и тогда счетчик не сможет обнулиться. Эта проблема обходится при помощи `weak_ptr`, так называемого слабого указателя. Класс `weak_ptr` похож на `shared_ptr`, но не участвует в подсчете ссылок.

# <a id='toc3_'></a>[Ещё о нововведениях C++11 и C++14](#toc0_)

## <a id='toc3_1_'></a>[Кортежи](#toc0_)

В старом стандарте имелся (и остается) шаблонный класс `std::pair` (`#include <utility>`). Создается обычно функцией `std::make_pair`. В новом стандарте появилось его расширение - кортеж (`std::tuple`).


    std::tuple<std::string, int, int> getUnitInfo(int id) {
        if (id == 0) return std::make_tuple("Elf", 60, 9);
        if (id == 1) return std::make_tuple("Dwarf", 80, 6);
        if (id == 2) return std::make_tuple("Orc", 90, 3);
        //...
    }
    
    int main() {
        auto ui0 = getUnitInfo(0);
        std::cout << "race: "<< std::get<0>(ui0) << ", "
            << "hp: " << std::get<1>(ui0) << ", "
            << "iq: " << std::get<2>(ui0) << "\n";
            
        std::string race1; int hp1; int iq1;
        std::tie(race1, hp1, iq1) = getUnitInfo(1);
        std::cout << "race: " << race1 << ", "
            << "hp: " << hp1 << ", "
            << "iq: " << iq1 << "\n";
    }

Создать можно либо конструктором `std::tuple<...>(...)` с явным указанием типов, либо функцией `std::make_tuple(...)`, которая сама выводит шаблонные параметры.

Распаковать объекты из кортежа в отдельные переменные можно:
- `std::get<N>(var_tuple)` по номеру `N` объекта в кортеже
- `std::tie(var1, ..., varN) = var_tuple;` - распаковать в список переменных
- В С++17 стало еще проще: `auto[var1, ..., varN] = var_tuple;`

**Задача**

Напишите шаблонную функцию `to_pair`, которая принимает произвольный `std::tuple` и два индекса внутри и возвращает `std::pair`, содержащий элементы переданного `std::tuple` с соответствующими индексами.

Пример:

    auto t = std::make_tuple(0, 3.5, "Hello");
    std::pair<double, char const *> p = to_pair<1,2>(t); 
    // p содержит 3.5 и "Hello"

    #include <tuple>

    // C++11
    // принимает std::tuple произвольного размера
    template<size_t I, size_t J, typename... Types>
    auto to_pair(std::tuple<Types...> const & t) -> 
    decltype(std::make_pair(std::get<I>(t), std::get<J>(t)))
    {
        return std::make_pair(std::get<I>(t), std::get<J>(t));
    }


- `Types` это "кортеж" типов, которые идут в `t`, из этой переменной шаблонной функцией `std::get` выбираем элементы по индексу
- использован синтаксис С++11, где при использовании `auto` и до обявления в коде `t`, через которую вычисляется возвращаемый тип нужно использовать `decltype` от выражения от `t` и других шаблонных параметров
    - в данном случае `decltype` передается выражение соответствующее временному объекту, поэтому его тип будет без ссылки
- в С++14 достаточно просто `auto`, помпилятор догадается о типе по выражению из `return`
```
    // С++14
    template<size_t I, size_t J, typename... Types>
    auto to_pair(std::tuple<Types...> const & t)
    {
        return std::make_pair(std::get<I>(t), std::get<J>(t));
    }
```
- т.к. нам не нужно менять аргумент, а только читать, и чтобы функция могла работать с константными объектами используем `const`, а чтобы не копировала аргументы, используем `const &`
    - к сожалению это не позволит функции работать по указателями, т.к. для этого надо в `std::get<I>(*t)`
- остался вопрос, как переменный шаблонный параметр `Types` неявно попадает в шаблон, ведь в коде мы вызываем просто `to_pair<1,2>(t)`
    - параметр `Types` компилятор может сам вывести по типу передаваемого значения (неочевидно, и нигде ранее не объяснялось, я то думал количество параметров в шаблоне и вызове должно быть четко связано с точностью до распаковки `...`, и при вызове `to_pair<1,2>(t)` шаблон должен получить пустой `Types`)

- еще фишка, т.к. `std::get` это функция, работающая только с `std::tuple`, поэтому нет необходимости усложнять проверкой типа в шаблоне на `std::tuple<Types...>` (если прилетит не кортеж, то не сработает вызов get и UB не будет), можно просто 
```
    // C++11
    // принимает std::tuple произвольного размера
    template<size_t I, size_t J, typename T>
    auto to_pair(T const & t) -> 
    decltype(std::make_pair(std::get<I>(t), std::get<J>(t)))
    {
        return std::make_pair(std::get<I>(t), std::get<J>(t));
    }
```

- немного более читаемым можно сделать так
```
    using std::make_pair; using std::get;
    template <int A, int B, typename T>
    auto to_pair(T t) -> decltype(make_pair(get<A>(t), get<B>(t))) {
        return make_pair(get<A>(t), get<B>(t));
    }
```

- вот такая простотень, а сколько ньюансов!

## <a id='toc3_2_'></a>[Константные выражения constexpr](#toc0_)

В старом стандарте вычисления на этапе компиляции касались только целочисленных констант. В новом стандарте к ним добавились числа с плавающей точкой, а также функции, которые **можно** вычислять на этапе компиляции

Для констант и функций времени компиляции.

    constexpr double accOfGravity = 9.8;                // константы времени компиляции
    constexpr double moonGravity = accOfGravity / 6;

    constexpr int pow(int x, int k)                     // функции, которые можно вычислять на этапе компиляции
    { return k == 0 ? 1 : x * pow(x, k - 1); }

    int data[pow(3, 5)] = {};

    struct Point {
        double x, y;
        constexpr Point(double x = 0, double y = 0)
            : x(x), y(y) {}
        constexpr double getX() const { return x; }
        constexpr double getY() const { return y; }
    };
    
    constexpr Point p(moonGravity, accOfGravity);       // будут вычислены и созданы на этапе компиляции
    constexpr auto x = p.getX();
    // в рантайме constexpr функции/методы также будут работать, но уже как обычные фукнции/методы

На функции, которые **можно** вычислять на этапе компиляции (`constexpr`) накладываются ряд ограничений:
- внутри таких функций не должны вызываться обычные не `constexpr`-функции
- в С++11 они были ограничены единственным `return`
- можно вызывать только `constexpr`-функции, в т.ч. саму себя (рекурсия возможна)
- дебажить их не получится (только отдельно как обычные функции)
    - определить, сработала ли функция при компиляции или в рантайме можно поставив в нее точку останова, если остановится, то это райнтайм
- C++14 все несколько упростилось, `constexpr`-функции стали ближе к обычными функциями, а конструктры получили более лояльные требования

Такими темпами в очередной версии в С++ компилятор встроят интерпретатор питона...

**Задача**

Напишите constexpr бинарные операторы +, - и * (скалярное произведение) для структуры Point.

Пример использования:

    constexpr size_t size = static_cast<size_t>(Point(2,4) * Point(4,3));
    int m[size]; // массив из 20 элементов

Вариант:

    // определение структуры Point уже подключено
    /* struct Point
    {
        constexpr Point(double x, double y) 
            : x(x), y(y) 
        {}

        double x = 0;
        double y = 0;
    };
    */

    // сложение
    constexpr Point operator+ (Point a, Point b) {
        return Point(a.x + b.x, a.y + b.y);
    }

    // вычитание
    constexpr Point operator- (Point a, Point b) {
        return Point(a.x - b.x, a.y - b.y);
    }

    // скалярное произведение
    constexpr double operator* (Point a, Point b) {
        return (a.x * b.x + a.y * b.y);
    }

## <a id='toc3_3_'></a>[Range-based for](#toc0_)

Новая синтаксическая конструкция для работы с последовательностями.

    int array[] = {1, 4, 9, 16, 25, 36, 49};
    
    int sum = 0;
    // по значению (сумма элементов)
    for (int x : array) {
        sum += x;
    }

    // по ссылке (удвоение элементов)
    for (int & x : array) {
        x *= 2;
    }

Применим к встроенным массивам, спискам инициализации, контейнерам из стандартной библиотеки и любым другим типам, для которых определены функции `begin()` и `end()`, возвращающие итераторы (об этом будет рассказано дальше).

Это просто синтаксический сахар. "Под капотом" реализуется самый обычный низкоуровневый цикл. Компилятор прекрасно оптимизирует и такие циклы, и те, что со счётчиком, и те, что без счётчика. Т.е. все они работают одинаково быстро.

## <a id='toc3_4_'></a>[Списки инициализации](#toc0_)

Возможность передать в функцию список значений.

Это позволяет создавать конструкторы, которые инициализируеются как массивы встроенных типов

    // в конструкторах массивов и других контейнеров
    template<typename T>
    struct Array {
        Array(std::initializer_list<T> list);
    };

    Array<int> primes = {2, 3, 5, 7, 11, 13, 17};

Также можно передавать такой список значений в функцию как один аргумент без распаковки и, например, проходить по нему `range based` форлупом

    // в обычных функциях
    int sum(std::initializer_list<int> list) {
        int result = 0;
        for (int x : list)
            result += x;
        return result;
    }
    
    int s = sum({1, 1, 2, 3, 5, 8, 13, 21});

**Замечания**
- у шаблонного класса `std::initializer_list` нет конструктора, который можно было бы использовать самостоятельно, создавать объекты этого класса можно только фигурными скобками `{}` как и списки инициализации встроенных типов
- все экземпляры этого класса неизменяемые 


## <a id='toc3_5_'></a>[Универсальная инициализация](#toc0_)

В новом стандарте фигурные скобки получили еще одно применение - **универсальная инициализация**.

Пусть есть структуры без конструктора и с конструктором:

    struct CStyleStruct {
        int x;
        double y;
    };
    struct CPPStyleStruct {
        CPPStyleStruct(int x, double y): x(x), y(y) {}
        int x;
        double y;
    };

    // C++03
    CStyleStruct s1 = {19, 72.0};   // инициализация
    CPPStyleStruct s2(19, 83.0);    // вызов конструктора

Теперь такие структуры можно инициализировать скобками вне зависимости есть конструктор или нет.

    // C++11
    CStyleStruct s1{19, 72.0};      // инициализация
    CPPStyleStruct s2{19, 83.0};    // вызов конструктора

А в таких ситуациях можно не указывать тип и возвращять список инициализации, компилятор по типу возвращяемого значения поймет, что нужен новый экземпляр этой структры

    // тип не обязателен
    CStyleStruct getValue() { return {6, 4.2}; }

## <a id='toc3_6_'></a>[std : : function](#toc0_)

Универсальный класс для хранения указателей на функции, указателей на методы и функциональных объектов (объекты с перегруженным оператором `()`).

Например, есть две реализации операций - в виде функции и функц.объекта:

    int mult (int x, int y) { return x * y; }

    // функциональный объект
    struct IntDiv {
        int operator()(int x, int y) const {    // const, т.к. состояние экз.структуры не меняется
            return x / y;
        }
    };

Мы хотим по условию выбирать операцию:

    std::function<int (int, int)> op;   // указатель на функцию с сигнатурой в параметрах шаблона

    if ( OP == ’*’ )
        op = &mult;             // можно без & - будет неявное приведение
    else if ( OP == ’/’)
        op = IntDiv();
    int result = op(7, 8);

Позволяет работать и с указателями на методы.


## <a id='toc3_7_'></a>[Лямбда-выражения](#toc0_)

По сути в С++ это такой синтаксический механизм создания функциональных объектов (объекты с перегруженным оператором `()`). Обычно их используют, чтобы сделать код более лаконичным и, по возможности, более читаемым (чтобы однострочная функция была читаемым образом определена прямо там где применяется, а не в отдельном файле `миллион_однострочных_одноразовых_функций.cpp`).

Лямбда-выражения

    std::function<int (int, int)> op =
        [](int x, int y) { return x / y; } // IntDiv
    
    // то же, но с указанием типа возвращаемого значения
    op = [](int x, int y) -> int { return x / y; }
    
    // С++14
    op = [](auto x, auto y) { return x / y; }

Можно захватывать локальные переменные и изменять их в лямбда-выражении, если они захвачены по ссылке, либо использовать для изменения других переменных, если захвачены по значению. Также можно вызывать методы `this`
    
    // захват по ссылке
    int total = 0;
    auto addToTotal = [&total](int x) { total += x; };
    
    // захват по значению (изменить не получится, т.к. это захваченная будет локальной в лямбде, но можно использовать)
    auto subTotal = [total](int & x) { x -= total ; };
    
    // Можно захватывать this (т.к. это всегда локальная переменная)
    auto callUpdate = [this](){ this->update(); };

**Задача**

Рассмотрим следующую функцию for_each.

    template<class F>
    void for_each_int(int * p, int * q, F f)
    {
        for ( ; p != q; ++p )
            f(*p);
    }

Определите переменную `square_fun`, хранящую безымянную функцию, которая возводит переданное число в квадрат.

    int m[10] = {1,2,3,4,5,6,7,8,9,10};

    auto square_fun = [](int & i) { i *= i; };

    // или так
    // #include <functional>
    // std::function<void (int &)> square_fun = [](int & i) { i *= i; };
    
    for_each_int(m, m + 10, square_fun); // теперь m = {1,4,9,16,25,36,49,64,81,100};




## <a id='toc3_8_'></a>[Различные виды захвата](#toc0_)

Могут быть разные типы захвата, в т.ч. смешанные:

    [], [x, &y], [&], [=], [&, x], [=, &z]

`[&, x]` все локальные переменные захватываются по ссылке, а `x` по значению  
`[=, &z]` все локальные переменные захватываются по значению, а `z` по ссылке

Перемещающий захват `[x = std::move(y)]` (только в C++14).

Не стоит использовать захват по умолчанию `[&]` (**все** локальные переменные, упомянутые в лямбда выражении будут захвачены по ссылке) или `[=]` (... по значению), т.к. это чревато ошибками, например:

- лямбда принимает инт аргумент и возвращает флаг делится ли оно на `d` (локальную переменную)
- но мы возвращаем не значение, а функциональный объект, когда он покинет функцию, `d` перестанет существовать, а когда следующим шагом пороизойдет попытка получить данные из функционального объекта по захваченной ссылке уже ничего не будет

```
    std::function<bool(int)> getFilter(Checker const& c) {
        auto d = c.getModulo();
        // захватывает ссылку на локальную переменную
        return [&] (int i) { return i % d == 0; }
    }
```
- аналогичная лямбда, но попробуем захватить `d` по значению
- но тут `d` уже не является локальной переменной, это поле класса, почему компилятор это скомпилировал? Потому что лямбда в классе и захватывает `this->d`, но **по значению**, а значит захватывается поле текущего экз.объекта `Checker.d`. К момент раскрытия лямбды его существование не гарантируется, т.к. ссылку `this` его никто не держит

```
    struct Checker {
        std::function<bool(int)> getFilter() const {
            // захватывает this, а не d
            return [=] (int x) { return x % d == 0; }
        }
        int d;
    };
```
- в этом примере, попытка захвата просто в виде `[d]` выдаст ошибку компиляции (`d` не локальная переменная), поэтому нужно либо прямо захватывать `this`, понимая что это не `d` и манипулировать уже с ним,  либо копировать значение поля `d` в локальную переменную и захватывать по значению

**Задача**

Рассмотрим следующую функцию find_if:

    template<class F>
    int * find_if(int * p, int * q, F f)
    {
        for ( ; p != q; ++p )
            if (f(*p))
                return p;
        return q;
    }

Определите переменную `gen_finder`, хранящую безымянную функцию, которая принимает массив значений типа int через два указателя и возвращает безымянную функцию, которая в свою очередь принимает значение типа int и проверяет, есть ли это значение в переданном массиве.

    int primes[5] = {2,3,5,7,11};

    int m[10] = {0,0,1,1,4,6,7,8,9,10};

    // принимает массив значений типа int через два указателя и возвращает безымянную 
    // функцию, которая в свою очередь принимает значение типа int и проверяет, есть ли
    // это значение в переданном массиве
    auto gen_finder = [](int * pr1, int * pr2) { 
        return [pr1, pr2](int x) -> bool { 
            for (int * curr = pr1; curr != pr2; ++curr)
                if (*curr == x)
                    return true;
            return false; 
        }; 
    };

    // first_prime будет указывать на число 7
    int * first_prime = find_if(m, m + 10, gen_finder(primes, primes + 5));

Что тут что:
- указатели на начало / конец массива primes захватываются по значению, т.к. они не должны меняться во всех вызовах
- в цикле заводится отдельная переменная счетчик, инициализирующаяся при каждом вызове указателем на начало
- `-> bool` просто для красоты, работает и без него

Другие красивые варианты:

    // используется find_if (если не найдено, то она досчитает до конца массива)
    const auto gen_finder = [](int* b, int* e) {
        return [b, e](int x) {
            auto compare = [x](int y) { return y == x; };
            return find_if(b, e, compare) != e;
        };
    };

## <a id='toc3_9_'></a>[Новые строковые литералы](#toc0_)

Проблема расширенного типа `wchar_t` в старом С++, то что его размер не определен стандартом и отличается в разных компиляторах. Поэтом добавили расширенные типы чаров под UTF, причем кодировки строк теперь можно определять литералом `u8/u/U`:


    u8"I’m a UTF-8 string."                         // char[]
    u"This is a UTF-16 string."                     // char16_t[]
    U"This is a UTF-32 string."                     // char32_t[]
    L"This is a wide-char string."                  // wchar_t[] остался для совместимости

Можно писать символы по коду в таблице юникода:

    u8"This is a Unicode Character: \u2018."
    u"This is a bigger Unicode Character: \u2018."
    U"This is a Unicode Character: \U00002018."

Можно писать сырые строки (когда сама строка - это то что в скобках или пользовательских разделятелях, причем в таком виде она может содержать слеши и кавычки без эскейпов)

    R"(The String Data \ Stuff " )"
    R"delimiter(The String Data \ Stuff " )delimiter"

Литералы и сырые строки можно совмещать

    LR"(Raw wide string literal \t (without a tab))"
    u8R"XXX(I’m a "raw UTF-8" string.)XXX"
    uR"*(This is a "raw UTF-16" string.)*"
    UR"(This is a "raw UTF-32" string.)"

## <a id='toc3_10_'></a>[Перечисления со строгой типизацией](#toc0_)

Перечисления со строгой типизацией

В C++ можно задавать перечисления при помощи конструкции enum. Например, в следующем коде определяется перечисление с именами цветов. Причём, BLACK будет соответствовать число 0, WHITE — 1, а RED — 2.

    enum COLORS 
    { 
       BLACK, WHITE, RED, 
       GREEN = 4, BLUE = 8
    }; 

Такие перечисления не являются типобезопасными — они представлены целыми числами, несмотря на то, что сами типы перечислений различны. Это позволяет, например, сравнивать значения из разных перечислений.  Другая проблема — это нарушение областей видимости: enum не задаёт области видимости, соответственно, все элементы перечисления "видны" снаружи (т.е. можно к ним обращаться не указывая имя перечисления).

В новом стандарте появился новый тип перечислений enum class, который не обладает этими недостатками.

    enum class COLORS 
    { 
       BLACK, WHITE, RED, 
       GREEN = 4, BLUE = 8
    }; 

Элементы такого перечисления невозможно **неявно** преобразовать в целые числа.
Например, следующий код не скомпилируется:


    int k = 0;

    while (k < COLORS::RED) {}

- если нужно, можно сделать явное приведение (`static_cast`), а в общем случае предполагается использовать такие значения просто как константы (но типобезопасные, которые в сравнениях и приведениях не дадут UB)

Тип классового перечисления теперь не зависит от реализации (по умолчанию int) и может быть задан явно:

    enum class DAYS : unsigned int 
    {
        MONDAY, TUESDAY, WEDNESDAY, THURSDAY,
        FRIDAY, SATURDAY, SUNDAY
    };

При этом для доступа к элементам такого перечисления необходимо будет указывать имя перечисления.

Не стоит использовать перечисления старого типа. Если такие перечисления уже есть в проекте, то их можно улучшить, указав явно соответствующий целочисленный тип. При этом сохранится совместимость с существующим кодом, т.к. элементы перечисления будут доступны без указания имени перечисления:

    enum COLORS : unsigned int 
    { 
        BLACK, WHITE, RED, 
        GREEN = 4, BLUE = 8
    }; 


# <a id='toc4_'></a>[Как работают rvalue-ссылки](#toc0_)

lvalue:
- Имя переменной или аргумента (даже если их тип rvalue-ссылка). Например std::cin.
- Вызов функции или оператора, тип возвращаемого значения которых является lvalue-ссылкой. Например std::cout << 1, ++it.
- Строковые литералы, например "Hello, world!".

rvalue:
- Нестроковый литерал, например 42, true, nullptr.
- Вызов функции или оператора, тип возвращаемого значения которых не является ссылочным
- Взятие адреса: &a.
- Вызов функции или оператора, тип возвращаемого значения которых является rvalue ссылкой. Например std::move(x).
a.m, где a — rvalue.


lvalue представляет собой объект, который занимает идентифицируемое место в памяти. rvalue все остальное, почти всегда они если надо lvalue неявно преобразуются в rvalue (когда они справа в присваиваниях, бинарных операциях и т.п.). Т.е. само по себе выражение `X()` это rvalue - оно **само по себе** не идентифицирует объект в памяти, но этот вызов может возвращать lvalue

    int globalvar = 20;
    int& foo() { return globalvar; } // возвращает ссылку, которая является lvalue
    ...
    foo() = 10; // так можно, но зачем?

Суть: lvalue-ссылка `&`, rvalue-ссылка `&&`. В С++11+ тип Т может быть либо бессылочным `T`, либо rvalue-ссылкой `T&`, либо lvalue-ссылкой `T&&`. С указателями аналогично: `T*, T*& T*&&`. rvalue стараются передаваться в функции как `T&&`, lvalue как `T&`. До этого были только просто `rvalue/lvalue`.

Это разделение появилось для / вследствие добавление move-семантики (переносящий оператор присваивания). rvalue "предпочитает" быть переданным в функцию c аргументом `T&&`, а lvalue — с `T&`. Это дает нам возможность поступать с объектами по-разному, что особенно важно для реализации конструкторов копирования и перемещения

Пример:

    #include <iostream>
    template <typename T>
    void summator(T && a, T && b) { a += b; }   // как бы ждет ссылку на rvalue

    int main() {
        int a = 1, b = 2;
        // подается ссылка на lvalue
        summator <int&> (a, b); // Правило склейки & (a и b - lvalue ссылки) и && (параметры функции) -> &
        // подается rvalue (ссылка на rvalue)
        summator <int> (1, 2); // Правило склейки && (1 и 2 - rvalue ссылки) и && (параметры функции) -> &&
        std::cout << a << std::endl; // Значение a изменилось, поскольку функция работала с параметрами как с lvalue ссылками;
    }

**Замечание**

Нужно отличать, чтоб не путать! `x` является именем переменной только при объявлении/определении (`X x;`). В остальных случаях `x` - это уже **выражение**. Выражение всегда имеет тип `T&`, где `T` - тип ПЕРЕМЕННОЙ `x`. Поэтому если создать переменную типа rval-ref `Type&& tmp = std::move(a);`, то выражение `tmp` будет иметь тип `Type&` и можно будет дальше писать `Type& b = tmp;`

## <a id='toc4_1_'></a>[Преобразование ссылок в шаблонах](#toc0_)

В старом стандарте, если при "разворачивании" шаблона выводится тип "ссылка на ссылку &&" это была ошибка компиляции. В новом стандарте появились т.н. "правила склейки"

“Склейка” ссылок:

    ∙ T& & → T& (lv)
    ∙ T& && → T& (lv)
    ∙ T&& & → T& (lv)
    ∙ T&& && → T&& (rv)

Такие правила склейки позволяют определить **универсальную ссылку** - это арг.функции, который передается как **rvalue-ссылка** и его тип при этом является **параметром** шаблона. 

    template<typename T>
    void foo(T && t) {}

- Если вызвать foo от lvalue типа A, то T = A&.
- Если вызвать foo от rvalue типа A, то T = A.

## <a id='toc4_2_'></a>[Как работает std : : move?](#toc0_)

Именно этот механизм использует std::move стандартной библиотеки, это шаблонная ф-я, использующая универсальную ссылку.

Определение std::move (основное):

    template<class T>
    typename remove_reference<T>::type&&
        move(T&& a)
    {
        typedef typename remove_reference<T>::type&& RvalRef;
        return static_cast<RvalRef>(a);
    }

- принимает 1 арг. - rvalue-ref на тип Т
- что (какой тип) она возвращает: используется метафункция станд.библиотеки `remove_reference`^
    - `A& -> A`, удаляет ссылку из типа A, если она есть, так что теперь можно приписать типу rvaval-ref
        - если этого не сделать, а просто приписать &&, то сработает правило склейки ссылок и тип получится `A& && -> A&` (lvalue-ref), т.е. функция не сможет принимать rvalue
    - в самой функции определен typedef при любом типе аргумента Т всегда будет иметь тип T&& (ссылка на rvalue)
    - в ретурне тип аргумента приводится к типу этого typedef
        - сам аргумент `a` при этом является уже локальной переменной в функции (т.е. lvalue `A&`), не важно какого типа был объект, передаваемый в функции изначально (rvalue/lvalue), поэтому эту переменную мы можем в функции изменять
- `remove_reference` (из `<type_traits>`) это шаблонная структура, которая независимо от аргументного типа хранит исходный тип без ссылки `_Tp` - просто написана в нескольких вариантах и нужный вариант выбирается компилятором при выведении типа `_Tp / _Tp& / _Tp&&` (т.н.частичная специализация шаблонов)

Замечаниe

std::move не выполняет никаких действий времени выполнения.

## <a id='toc4_3_'></a>[std : : move для lvalue](#toc0_)

Вызываем std::move для **lvalue объекта**.

    X x;
    x = std::move(x); // справа в скобках выражение x имеет тип X&

Тип T в `std::move` выводится как X&. Примерно таким образом (подставим `X&` в определение `std::move`):

    typename remove_reference<X&>::type&&
    move(X& && a)
    {
    typedef typename remove_reference<X&>::type&& RvalRef;
    return static_cast<RvalRef>(a);
    }

После склейки ссылок получаем:

    X&& move(X& a)
    {
    return static_cast<X&&>(a);
    }

Т.е. в результате не происходит никакого копирования, передается значение по lvalue ссылке и внутри происходит статик каст к rvalue ссылке, поэтому объект прекрасно присваивается в выражении `x = std::move(x);`

## <a id='toc4_4_'></a>[std : : move для rvalue](#toc0_)

Вызываем std::move для временного объекта.

    X x = std::move(X()); // X() это выражение, кот. возвращает сам объект, т.е. просто rvalue вариант типа

Тип T в `std::move`  выводится как X:

    typename remove_reference<X>::type&&    // убирает ссылки (которых нет) и добавляет &&
    move(X&& a)
    {
    typedef typename remove_reference<X>::type&& RvalRef;
    return static_cast<RvalRef>(a);
    }

После склейки ссылок получаем:

    X&& move(X&& a)
    {
    return static_cast<X&&>(a);
    }

Точно также возвр. rval-ref, на объект, который вернуло выражение `X()`, т.е. можно присваивать как rvalue

## <a id='toc4_5_'></a>[Perfect forwading](#toc0_)

Техника (прием) программирования, применяется, если мы хотим сохранить способ передачи аргумента в другую функцию таким же, как этот аргумент пришел в данную функцию (как rvalue, либо как lvalue).

Например, мы пишем функцию, которая создает объект типа `T` (значение) и возвращает указатель `unique_ptr` на этот объект (значение). Пусть у `T` нет конструктора по умолчанию, а есть только конструктор от 1 аргумента, т.е. при создании нового объекта/значения типа T мы должны передать аргумент в этот конструктор, и мы не хотим его копировать.

Такая ф-я может быть шаблонной с 2 шаблонными параметрами T и аргумент конструктора. Тогда в зависимости от того, как передается параметр конструктора нам нужны две перегрузки этой шаблонной функции:

    // для lvalue
    template<typename T, typename Arg>
    unique_ptr<T> make_unique(Arg & arg) {
    return unique_ptr<T>(new T(arg));
    }
    
    // для rvalue
    template<typename T, typename Arg>
    unique_ptr<T> make_unique(Arg && arg) {
    return unique_ptr<T>(new T(std::move(arg)));
    }

- в случае rvalue варианта перегрузки `arg` будет локальной переменной, поэтому его нужно обернуть в `std::move`

`std::forward` позволяет записать это одной функцией.

    template<typename T, typename Arg>
    unique_ptr<T> make_unique(Arg&& arg) {
    return unique_ptr<T>(
    new T(std::forward<Arg>(arg)));
    }

- используется тот факт, что когда rvalue параметр шаблона, то это универсальная ссылка. `std::forward` просто вернет ссылку, если его шаблонный параметр это lvalue или вызовет move если rvalue. Как он это определяет?

## <a id='toc4_6_'></a>[Как работает std : : forward?](#toc0_)

Определение `std::forward`:

    template<class S>
    S&& forward(typename remove_reference<S>::type& a)
    {
        return static_cast<S&&>(a);
    }

- кажется, что она всегда возвр. rval-ref, но это если не учитывать склейку ссылок, финальный тип зависит от нее (комбинации типа аргумента в шаблоне и типа передаваемого объекта)!

Замечаниe

`std::forward` не выполняет никаких действий времени выполнения.

## <a id='toc4_7_'></a>[std : : forward для lvalue и rvalue](#toc0_)

**std::forward для lvalue**

    X x;
    auto p = make_unique<A>(x); // Arg = X&

    unique_ptr<A> make_unique(X& && arg) {
        return unique_ptr<A>(new A(std::forward<X&>(arg))); // склейка ссылок
    }

    X& && forward(remove_reference<X&>::type& a) {
        return static_cast<X& &&>(a);
    }

После склейки ссылок:

    unique_ptr<A> make_unique(X& arg) {
        return unique_ptr<A>(new A(std::forward<X&>(arg)));
    }
    
    X& forward(X& a) {
        return static_cast<X&>(a);
    }

**std::forward для rvalue**

    auto p = make_unique<A>(X()); // Arg = X
    unique_ptr<A> make_unique(X&& arg) {
        return unique_ptr<A>(new A(std::forward<X>(arg)));
    }

    X&& forward(remove_reference<X>::type& a) {
        return static_cast<X&&>(a);
    }

После склейки ссылок (которой не происходит, т.к. не надо):

    unique_ptr<A> make_unique(X&& arg) {
        return unique_ptr<A>(new A(std::forward<X>(arg)));
    }

    X&& forward(X& a) {
        return static_cast<X&&>(a);
    }

## <a id='toc4_8_'></a>[Variadic templates + rvalue reference](#toc0_)

Можно применить std::forward для списка параметров.

    template<typename T, typename ...Args>                  // понятно
    std::unique_ptr<T> make_unique(Args&&... args) {        // понятно
        return std::unique_ptr<T>(
            new T(std::forward<Args>(args)...));            // в коде функции!
    }

- `...` в коде функции - берет предыдущую инструкцию, список типов и аргументов функции, проходит по этим спискам и подставляет их все в инструкцию. Такая как бы распаковка аргументов (в увязке с типами)

Теперь make_unique работает для произвольного числа аргументов.

    auto p = make_unique<Array<string>>(10, string("Hello"));

- аргументы тут будут rvalue-ref, поэтому если есть к.пермещ., то копироваться не будут

**Задача**

Напишите функцию apply, которая принимает некоторую функцию / функциональный объект, а так же аргументы для вызова этого объекта, и вызывает его, используя perfect forwarding.


    #include <utility>

    template<typename Fun, typename ...Args>
    auto apply(Fun fun, Args&&... args) -> 
    decltype(fun(std::forward<Args>(args)...))
    {
        return fun(std::forward<Args>(args)...);
    }

В чем был затык:
- `Fun fun` ! нельзя вызывать тип: `return Fun(...)`

## <a id='toc4_9_'></a>[Резюме по std::move и std::forward](#toc0_)

**std::move нужно применять:**

когда хотите явно передать значение по rvalue-ссылке:

    X x;
    foo(std::move(x)); // передача rvalue-ссылки

или когда хотите передать rvalue-ссылку - аргумент функции в другую функцию:

    void foo(X && x)
    {
        bar(std::move(x)); // передача rvalue-ссылки
    }


**std::forward нужно применять:**

только тогда, когда вы хотите перенаправить аргумент функции - универсальную ссылку в другую функцию (например, в шаблоне):

    template<class T>
    void foo(T && t)
    {
        bar(std::forward<T>(t)); // перенаправление
    }